In [10]:
import pandas as pd
import numpy as np
import torch 


hrs = pd.read_csv("life_expectancy_CleanedHRSdata.csv")
pd.set_option('display.max_columns', None)

print(hrs.columns)
hrs

Index(['Unnamed: 0', 'hhidpn', 'wave', 'mstat', 'cendiv', 'gender', 'rahispan',
       'raracem', 'iwbeg', 'dage_m', 'dage_y', 'raedyrs', 'rarelig', 'ravetrn',
       'agey_m', 'shlt', 'shltc', 'depres', 'effort', 'sleepr', 'cesd', 'bmi',
       'smokev', 'smoken', 'drinkn', 'hibp', 'diab', 'cancr', 'lung', 'heart',
       'strok', 'psych', 'arthr', 'conde', 'cogtot', 'slfmem', 'pstmem',
       'spcfac', 'hsptim', 'puff', 'puffpos', 'timwlk', 'hatotb', 'iearn',
       'isret', 'covs', 'hiltc', 'lbrf', 'logiearn', 'logisret', 'loghspti',
       'loghatotb', 'id', 'nt', 'n2'],
      dtype='object')


,Unnamed: 0,hhidpn,wave,mstat,cendiv,gender,rahispan,raracem,iwbeg,dage_m,dage_y,raedyrs,rarelig,ravetrn,agey_m,shlt,shltc,depres,effort,sleepr,cesd,bmi,smokev,smoken,drinkn,hibp,diab,cancr,lung,heart,strok,psych,arthr,conde,cogtot,slfmem,pstmem,spcfac,hsptim,puff,puffpos,timwlk,hatotb,iearn,isret,covs,hiltc,lbrf,logiearn,logisret,loghspti,loghatotb,id,nt,n2
0,1,3010,3,1.married,9.pacific,1.male,0.not hispanic,1.white/caucasian,13345,931,77,12,1.protestant,0.no,60,3.good,-1,1.yes,1.yes,1.yes,3,28.000000,0.no,0.no,1,0.no,0.no,0.no,0.no,1.yes,0.no,0.no,0.no,1,35,4.fair,2.same,0.no,0,NaN,NaN,NaN,490500.0,4000.0,0.0,1.yes,1.yes,4.partly retired,8.294049,0.000000,0.0,13.103181,2,123702,7
1,2,3010,6,1.married,9.pacific,1.male,0.not hispanic,1.white/caucasian,15445,931,77,12,1.protestant,0.no,66,3.good,-1,0.no,0.no,0.no,1,28.299999,0.no,0.no,0,0.no,0.no,0.no,0.no,1.yes,0.no,0.no,0.no,1,31,3.good,2.same,0.no,0,NaN,NaN,NaN,704000.0,10000.0,13728.0,0.no,1.yes,4.partly retired,9.210340,9.527193,0.0,13.464534,3,123702,7
2,3,3010,7,1.married,9.pacific,1.male,0.not hispanic,1.white/caucasian,16267,931,77,12,1.protestant,0.no,68,3.good,0,0.no,0.no,0.no,0,26.600000,0.no,0.no,0,0.no,0.no,0.no,0.no,1.yes,0.no,0.no,0.no,1,18,3.good,2.same,0.no,0,NaN,NaN,NaN,756000.0,6000.0,15600.0,0.no,1.yes,4.partly retired,8.699514,9.655026,0.0,13.535797,4,123702,7
3,4,3010,8,1.married,9.pacific,1.male,0.not hispanic,1.white/caucasian,16875,931,77,12,1.protestant,0.no,70,3.good,0,0.no,0.no,0.no,0,27.100000,0.no,0.no,0,0.no,0.no,0.no,0.no,1.yes,0.no,0.no,0.no,1,20,4.fair,2.same,0.no,0,490.0,1.standing,2.78,914000.0,0.0,14040.0,0.no,1.yes,5.retired,0.000000,9.549665,0.0,13.725586,5,123702,7
4,6,3010,10,1.married,9.pacific,1.male,0.not hispanic,1.white/caucasian,18520,931,77,12,1.protestant,0.no,74,3.good,0,0.no,0.no,0.no,0,24.000000,0.no,0.no,0,0.no,0.no,0.no,0.no,1.yes,0.no,0.no,0.no,1,17,4.fair,3.worse,0.no,0,330.0,1.standing,3.10,1240000.0,0.0,16644.0,0.no,1.yes,5.retired,0.000000,9.719805,0.0,14.030622,7,123702,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4706,13126,213435010,4,7.widowed,5.s atlantic,2.female,0.not hispanic,1.white/caucasian,14045,1082,90,12,2.catholic,0.no,74,3.good,.p,0.no,0.no,0.no,0,24.900000,1.yes,0.no,2,1.yes,0.no,0.no,0.no,0.no,0.no,0.no,0.no,1,24,3.good,2.same,0.no,0,NaN,NaN,NaN,87000.0,0.0,8856.0,0.no,1.yes,5.retired,0.000000,9.088850,0.0,11.373663,107639,123702,7
4707,13127,213435010,5,7.widowed,5.s atlantic,2.female,0.not hispanic,1.white/caucasian,14715,1082,90,12,2.catholic,0.no,75,3.good,0,0.no,0.no,0.no,0,25.700001,1.yes,0.no,2,1.yes,0.no,0.no,0.no,0.no,0.no,0.no,0.no,1,24,3.good,2.same,0.no,0,NaN,NaN,NaN,90000.0,0.0,8400.0,0.no,1.yes,5.retired,0.000000,9.035987,0.0,11.407565,107640,123702,7
4708,13128,213435010,6,7.widowed,5.s atlantic,2.female,0.not hispanic,1.white/caucasian,15475,1082,90,12,2.catholic,0.no,78,3.good,0,0.no,0.no,0.no,0,27.100000,1.yes,0.no,2,1.yes,0.no,0.no,0.no,0.no,0.no,0.no,0.no,1,23,3.good,3.worse,0.no,1,NaN,NaN,NaN,84750.0,0.0,9600.0,0.no,1.yes,5.retired,0.000000,9.169519,0.0,11.347461,107641,123702,7
4709,13129,213435010,7,7.widowed,5.s atlantic,2.female,0.not hispanic,1.white/caucasian,16206,1082,90,12,2.catholic,0.no,80,3.good,0,0.no,0.no,0.no,0,25.400000,1.yes,0.no,3,1.yes,0.no,0.no,0.no,0.no,0.no,0.no,1.yes,2,25,3.good,3.worse,0.no,1,NaN,NaN,NaN,111000.0,0.0,9912.0,0.no,1.yes,5.retired,0.000000,9.201502,0.0,11.617286,107642,123702,7


In [15]:
columns_to_drop = ['Unnamed: 0', 'nt','n2','dage_y']

hrs_cleaned = hrs.drop(columns=columns_to_drop, errors='ignore')
    # Replace empty strings with NaN to handle both empty strings and NaN values uniformly
hrs_cleaned = hrs_cleaned.replace('', np.nan)
    
    # Drop rows with any NaN values
hrs_cleaned = hrs_cleaned.dropna()
    
hrs_cleaned




,hhidpn,wave,mstat,cendiv,gender,rahispan,raracem,iwbeg,dage_m,raedyrs,rarelig,ravetrn,agey_m,shlt,shltc,depres,effort,sleepr,cesd,bmi,smokev,smoken,drinkn,hibp,diab,cancr,lung,heart,strok,psych,arthr,conde,cogtot,slfmem,pstmem,spcfac,hsptim,puff,puffpos,timwlk,hatotb,iearn,isret,covs,hiltc,lbrf,logiearn,logisret,loghspti,loghatotb,id
3,3010,8,1.married,9.pacific,1.male,0.not hispanic,1.white/caucasian,16875,931,12,1.protestant,0.no,70,3.good,0,0.no,0.no,0.no,0,27.100000,0.no,0.no,0,0.no,0.no,0.no,0.no,1.yes,0.no,0.no,0.no,1,20,4.fair,2.same,0.no,0,490.0,1.standing,2.78,914000.0,0.0,14040.0,0.no,1.yes,5.retired,0.0,9.549665,0.000000,13.725586,5
4,3010,10,1.married,9.pacific,1.male,0.not hispanic,1.white/caucasian,18520,931,12,1.protestant,0.no,74,3.good,0,0.no,0.no,0.no,0,24.000000,0.no,0.no,0,0.no,0.no,0.no,0.no,1.yes,0.no,0.no,0.no,1,17,4.fair,3.worse,0.no,0,330.0,1.standing,3.10,1240000.0,0.0,16644.0,0.no,1.yes,5.retired,0.0,9.719805,0.000000,14.030622,7
7,3020,8,1.married,9.pacific,2.female,0.not hispanic,1.white/caucasian,16875,925,16,1.protestant,0.no,67,3.good,0,0.no,0.no,0.no,0,24.700001,0.no,0.no,0,0.no,0.no,1.yes,0.no,0.no,0.no,1.yes,1.yes,3,26,4.fair,3.worse,0.no,0,400.0,1.standing,2.37,914000.0,0.0,6000.0,0.no,1.yes,5.retired,0.0,8.699514,0.000000,13.725586,11
8,3020,10,1.married,9.pacific,2.female,0.not hispanic,1.white/caucasian,18520,925,16,1.protestant,0.no,72,4.fair,0,0.no,0.no,0.no,0,25.700001,0.no,0.no,0,0.no,0.no,1.yes,0.no,0.no,0.no,1.yes,1.yes,3,25,3.good,2.same,1.yes,1,370.0,1.standing,2.47,1240000.0,0.0,8400.0,0.no,1.yes,5.retired,0.0,9.035987,0.000000,14.030622,13
9,3020,12,7.widowed,9.pacific,2.female,0.not hispanic,1.white/caucasian,19858,925,16,1.protestant,0.no,75,4.fair,0,0.no,0.no,0.no,0,23.200001,0.no,0.no,0,1.yes,0.no,1.yes,0.no,1.yes,0.no,1.yes,1.yes,5,24,3.good,2.same,1.yes,2,330.0,1.standing,5.68,1070000.0,0.0,15600.0,0.no,1.yes,5.retired,0.0,9.655026,0.693147,13.883169,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4692,213415020,9,1.married,8.mountain,1.male,0.not hispanic,1.white/caucasian,17698,1093,16,1.protestant,1.yes,83,4.fair,1,0.no,1.yes,0.no,1,25.799999,1.yes,0.no,0,0.no,0.no,0.no,0.no,0.no,0.no,0.no,1.yes,1,14,4.fair,2.same,1.yes,1,180.0,1.standing,4.06,400000.0,0.0,14400.0,0.no,0.no,5.retired,0.0,9.574984,0.000000,12.899220,107542
4693,213415020,11,"2.married,spouse absent",8.mountain,1.male,0.not hispanic,1.white/caucasian,19281,1093,16,1.protestant,1.yes,87,3.good,0,1.yes,0.no,0.no,4,24.100000,1.yes,0.no,0,0.no,0.no,0.no,0.no,0.no,0.no,0.no,0.no,1,9,4.fair,3.worse,0.no,0,260.0,1.standing,13.06,160000.0,0.0,1125.0,0.no,0.no,5.retired,0.0,7.025538,0.000000,11.982929,107543
4700,213420010,8,1.married,8.mountain,1.male,0.not hispanic,1.white/caucasian,16967,1049,13,1.protestant,0.no,78,4.fair,1,1.yes,1.yes,1.yes,5,28.000000,0.no,0.no,2,1.yes,0.no,1.yes,0.no,0.no,0.no,0.no,1.yes,3,20,4.fair,3.worse,0.no,0,380.0,1.standing,3.35,1955000.0,0.0,11520.0,0.no,0.no,5.retired,0.0,9.351840,0.000000,14.485901,107574
4705,213421010,8,4.separated,8.mountain,1.male,0.not hispanic,1.white/caucasian,16967,1053,4,1.protestant,0.no,81,5.poor,0,0.no,0.no,0.no,0,26.600000,1.yes,0.no,0,0.no,0.no,0.no,1.yes,0.no,0.no,0.no,4.disp prev record and no cond,1,20,3.good,2.same,1.yes,0,130.0,1.standing,4.30,300.0,0.0,8904.0,0.no,0.no,5.retired,0.0,9.094255,0.000000,5.703783,107592
